In [4]:
import pandas as pd
import mongoengine as me
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

# Conexion con SQL Server y MongoDB

In [5]:
#conectando a MongoDB
me.connect('ko_database_main')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [6]:
connection_string = "DRIVER={ODBC Driver 11 for SQL Server};SERVER=XXXXXX;DATABASE=KO_DATABASE_MAIN;UID=sa;PWD=XXX"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
make_query = lambda x: pd.read_sql(x, engine)

### test de la conexion a SQL SERVER

In [19]:
make_query('select * from GAME_OPTIONS').head(5)

,MaintenanceMode,CharacterSelectLogin,OpenOTP,AutoRegister,FreeLimit,TotalUserLimit,ServerIPAddress
0,0,1,0,0,300,300,18.190.138.145


### test de la conexion a mongodb

In [8]:
class dummy(me.Document):
    hi = me.StringField()

for document in dummy.objects:
    print(document.hi)

bye


# Mapeando SQL types hacia MongoDB documents

In [20]:
make_query('SELECT distinct DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS')

,DATA_TYPE
0,char
1,datetime
2,int
3,smallint
4,tinyint
5,varchar


In [10]:
# mapeo hacia documentDB e importacion de los documentos requeridos.
from mongoengine import StringField, DateTimeField, IntField
map_data_types = {
    'char' : 'StringField',
    'datetime' : 'DateTimeField',
    'int' : 'IntField',
    'smallint' : 'IntField',
    'tinyint' : 'IntField',
    'varchar' : 'StringField'
}

In [18]:
#Obtenemos las tablas
tables_names = make_query("SELECT TABLE_NAME FROM KO_DATABASE_MAIN.INFORMATION_SCHEMA.TABLES;")
tables_names.head(10)

,TABLE_NAME
0,GM_MAINTENANCE_LOGIN
1,SERVER_CLEAN
2,SERVER_SETTINGS
3,GAME_OPTIONS
4,BEGINNER_SETTINGS
5,GAME_MASTER_SETTINGS
6,ACCOUNT_LOGIN_CONTROL


In [17]:
# query para obtener las columnas y sus tipos de datos para cada tabla.
tabla_test_info =make_query("SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'BEGINNER_SETTINGS'")
tabla_test_info.head(5)

,COLUMN_NAME,DATA_TYPE
0,ServerNo,smallint
1,BeginnerType,tinyint
2,BeginnerDesc,varchar


# Escritura de MongoEngine Documents

In [13]:
!del MongoEngineDocuments.py
with open('MongoEngineDocuments.py','a+') as my_python_file:
    print(f'from mongoengine import Document, ', end='', file=my_python_file)
    print(*set(map_data_types.values()),sep=', ', end='\n\n', file=my_python_file)
    for TABLE_NAME in tables_names.TABLE_NAME:
        print(f'class {TABLE_NAME}(Document):', file=my_python_file)
        for a,b in make_query(
                f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{TABLE_NAME}'"
        ).iterrows():
            print(f'    {b.COLUMN_NAME} = {map_data_types[b.DATA_TYPE]}()', file=my_python_file)
        print(f"    meta = {{'collection': '{TABLE_NAME}'}}", end='\n\n', file=my_python_file)

In [14]:
# ahora cargamos todos los documentos creados
from MongoEngineDocuments import *

# Trasladando los datos desde MSSQL hacia DocumentDB

In [15]:
for TABLE_NAME in tables_names.TABLE_NAME:
    BASE_QUERY = f'Select * from {TABLE_NAME}'
    data_records = make_query(BASE_QUERY).to_dict("records")
    records_instances = [eval(TABLE_NAME)(**data) for data in data_records]
    if records_instances:
        print(f'Loading table  {TABLE_NAME}')
        eval(TABLE_NAME).objects.insert(records_instances, load_bulk=False)
    else:
        print(f'Coleccion vacia: {TABLE_NAME}. No generara problemas futuros.')

Coleccion vacia: GM_MAINTENANCE_LOGIN. No generara problemas futuros.
Loading table  SERVER_CLEAN
Loading table  SERVER_SETTINGS
Loading table  GAME_OPTIONS
Loading table  BEGINNER_SETTINGS
Loading table  GAME_MASTER_SETTINGS
Coleccion vacia: ACCOUNT_LOGIN_CONTROL. No generara problemas futuros.
